<a href="https://colab.research.google.com/github/ayakow1/ttic31220-japanparliament-analysis/blob/main/BERTopic_hierarchy%2Bcommittee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install bertopic

In [59]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
%%capture
!pip install googletrans==3.1.0a0

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import sqlite3
import pandas as pd
from bertopic import BERTopic
import numpy as np
from googletrans import Translator

In [3]:
# Import Data
use_raw = True
if use_raw:
    name = "raw_speech"
else:
    name = "speech"

In [4]:
conn = sqlite3.connect(f'/content/drive/MyDrive/議事録/{name}.db')
all = pd.read_sql_query(f'''SELECT * FROM {name} WHERE house ='衆議院' AND speech_date >= '2020-01-01' AND speech_date <= '2023-04-31' ''', conn)
docs = all['speech'].to_list()
conn.close()

In [5]:
topic_model = BERTopic.load("/content/drive/MyDrive/議事録/BERTopic/model_shuugiin_3y")

In [6]:
topic_model.get_topic_info()[:60]

,Topic,Count,Name
0,-1,34792,-1_です_という_こと_ない
1,0,3135,0_ください_でしょ_よろしい_大臣
2,1,1425,1_雇用_賃上げ_賃金_最低賃金
3,2,1386,2_接種_ワクチン_副反応_高齢者
4,3,1006,3_消費税_課税_税制_法人税
5,4,859,4_病院_医療_医師_患者
6,5,725,5_ありがとう_ござい_まし_拍手
7,6,669,6_予算_補正予算_予備費_経費
8,7,641,7_学校_教員_学級_研修
9,8,629,8_原発_原子力_規制_稼働


In [7]:
docs = all['speech'].to_list()
len(docs)

71357

In [8]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

100%|██████████| 450/450 [00:07<00:00, 57.43it/s]


In [30]:
trans=Translator()

topic_lst = topic_model.get_topic_info()[1:21]["Name"].to_list()
topic_lst_t = ['_'.join([trans.translate(k).text for k in t.split('_')[:-1]]) for t in topic_lst]
topic_lst_td = {i:k for i,k in enumerate(topic_lst_t)}
topic_lst_td

{0: '0_please give me_right_All right',
 1: '1_employment_wage increase_wage',
 2: '2_inoculation_vaccination_side reaction',
 3: '3_sale tax_taxation_tax system',
 4: '4_hospital_medical care_doctor',
 5: '5_thank you_thank you_Better',
 6: '6_budget_Supplementary budget_Reserve fund',
 7: '7_School_teacher_class',
 8: '8_nuclear power plant_nuclear power_regulation',
 9: '9_Ukraine_Russia_aggression',
 10: '10_municipality_place_region',
 11: '11_Infect_Infected_spread of infection',
 12: '12_housing_rent_empty house',
 13: '13_agriculture_farmland_farmer',
 14: '14_Digital_digitalization_society',
 15: '15_novel coronavirus_COVID-19 infection_Infect',
 16: '16_transportation_the way_railroad',
 17: '17_carbon_carbon neutral_discharge',
 18: '18_pension_load_Insurance fee',
 19: '19_declaration of emergency_lifted_extend'}

In [31]:
topic_model.set_topic_labels(topic_lst_td)

In [34]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics,  top_n_topics=20, 
                                custom_labels=True, width=800, height=150)

In [35]:
classes = all["committee"].to_list()

In [36]:
# Reduce the number of committees in the dataframe
num_comm = 10
target_comm = all["committee"].value_counts()[:num_comm].index.tolist()

In [37]:
doc_info = topic_model.get_document_info(docs)

In [38]:
allm = pd.concat([all.reset_index(drop=True),doc_info],axis=1,ignore_index=True)
allm.columns=list(all.columns)+list(doc_info.columns)

In [39]:
allm

,id,house,committee,vol,speech_date,speaker,party,speech,morpheme,Document,Topic,Name,CustomName,Top_n_words,Probability,Representative_document
0,120105261X00420200131_006,衆議院,予算委員会,第4号,2020-01-31,中山泰秀,自由民主党・無所属の会,早速 で は ござい ます が 、 質疑 に 移ら せ て いただき たい と 思い ます ...,副詞助詞類接続 助動詞* 助詞係助詞 動詞自立 助動詞* 助詞接続助詞 記号読点 名詞サ変接...,早速 で は ござい ます が 、 質疑 に 移ら せ て いただき たい と 思い ます ...,19,19_緊急事態宣言_解除_延長_蔓延,19_declaration of emergency_lifted_extend,緊急事態宣言 - 解除 - 延長 - 蔓延 - 発令 - 措置 - 緊急事態 - 発出 - ...,0.223309,False
1,120105261X00420200131_011,衆議院,予算委員会,第4号,2020-01-31,中山泰秀,自由民主党・無所属の会,特に 、 国民 の 皆様 は 、 目 に 見え ない この ウイルス 、 病気 に対して 非...,副詞一般 記号読点 名詞一般 助詞連体化 名詞一般 助詞係助詞 記号読点 名詞一般 助詞格助...,特に 、 国民 の 皆様 は 、 目 に 見え ない この ウイルス 、 病気 に対して 非...,-1,-1_です_という_こと_ない,-1_です_という_こと_ない,です - という - こと - ない - ます - いる - けれども - これ - 思い ...,0.000000,False
2,120105261X00420200131_013,衆議院,予算委員会,第4号,2020-01-31,中山泰秀,自由民主党・無所属の会,そして 同時に 、 これ は 大量 の 旅行客 の キャンセル という もの が 出 て お...,接続詞* 副詞一般 記号読点 名詞代名詞 助詞係助詞 名詞形容動詞語幹 助詞連体化 名詞固有...,そして 同時に 、 これ は 大量 の 旅行客 の キャンセル という もの が 出 て お...,56,56_観光_観光産業_インバウンド_観光庁,56_観光_観光産業_インバウンド_観光庁,観光 - 観光産業 - インバウンド - 観光庁 - 旅行 - 観光業 - 需要 - got...,0.399022,False
3,120105261X00420200131_015,衆議院,予算委員会,第4号,2020-01-31,中山泰秀,自由民主党・無所属の会,しっかり と し た 経済 対策 、 これ は かなり の 規模感 も 重要 に なっ て ...,副詞助詞類接続 助詞格助詞 動詞自立 助動詞* 名詞一般 名詞サ変接続 記号読点 名詞代名詞...,しっかり と し た 経済 対策 、 これ は かなり の 規模感 も 重要 に なっ て ...,348,348_中古住宅_経済_マイナポータル_バンク,348_中古住宅_経済_マイナポータル_バンク,中古住宅 - 経済 - マイナポータル - バンク - 地銀 - 研究者 - 年度 - 剰余...,1.000000,False
4,120105261X00420200131_017,衆議院,予算委員会,第4号,2020-01-31,中山泰秀,自由民主党・無所属の会,こう いっ た 世界 的 な 、 また 日本 の 中 でも 不安 が ある 中 で 、 国民...,副詞助詞類接続 動詞自立 助動詞* 名詞一般 名詞接尾 助動詞* 記号読点 接続詞* 名詞固...,こう いっ た 世界 的 な 、 また 日本 の 中 でも 不安 が ある 中 で 、 国民...,67,67_サイバー攻撃_サイバー_サイバーセキュリティー_攻撃,67_サイバー攻撃_サイバー_サイバーセキュリティー_攻撃,サイバー攻撃 - サイバー - サイバーセキュリティー - 攻撃 - セキュリティー - 捜...,0.673350,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71352,121104183X00120230302_032,衆議院,憲法審査会,第1号,2023-03-02,三木圭恵,日本維新の会,日本維新の会 の 三木圭恵 で ござい ます 。 予算 審議 の 最中 は 憲法審査会 を ...,名詞固有名詞 助詞連体化 名詞固有名詞 助動詞* 助動詞* 助動詞* 記号句点 名詞一般 名...,日本維新の会 の 三木圭恵 で ござい ます 。 予算 審議 の 最中 は 憲法審査会 を ...,-1,-1_です_という_こと_ない,-1_です_という_こと_ない,です - という - こと - ない - ます - いる - けれども - これ - 思い ...,0.000000,False
71353,121104183X00120230302_033,衆議院,憲法審査会,第1号,2023-03-02,國重徹,公明党,先 ほど 吉田 委員 の 方 から も 言及 が あり まし た けれども 、 今 、 性的...,名詞一般 助詞副助詞 名詞固有名詞 名詞一般 助詞連体化 名詞非自立 助詞格助詞 助詞係助詞...,先 ほど 吉田 委員 の 方 から も 言及 が あり まし た けれども 、 今 、 性的...,158,158_同性婚_婚姻_lgbt_同性愛,158_同性婚_婚姻_lgbt_同性愛,同性婚 - 婚姻 - lgbt - 同性愛 - 性的指向 - 性的マイノリティー - 同性 ...,1.000000,True
71354,121104183X00120230302_034,衆議院,憲法審査会,第1号,2023-03-02,山下貴司,自由民主党・無所属の会,私 は 、 新藤 筆頭 幹事 が 指摘 し た 論点 の うち 、 憲法九条 について も ...,名詞代名詞 助詞係助詞 記号読点 名詞固有名詞 名詞一般 名詞一般 助詞格助詞 名詞サ変接続...,私 は 、 新藤 筆頭 幹事 が 指摘 し た 論点 の うち 、 憲法九条 について も ...,181,181_憲法_改憲_憲法九条_議論,181_憲法_改憲_憲法九条_議論,憲法 - 改憲 - 憲法九条 - 議論 - 九条 - 自衛隊 - 自民党 - 憲法審査会 -...,1.000000,True
71355,121104183X00120230302_035,衆議院,憲法審査会,第1号,2023-03-02,新垣邦男,立憲民主党・無所属,先 ほど 同僚 の 吉田 はるみ 委員 から 、 平時 において も 国会 は 機能 し て...,名詞一般 助詞副助詞 名詞一般 助詞連体化 名詞固有名詞 名詞固有名詞 名詞一般 助詞格助詞...,先 ほど 同僚 の 吉田 はるみ 委員 から 、 平時 において も 国会 は 機能 し て...,30,30_国会_法案_審議_法律,30_国会_法案_審議_法律,国会 - 法案 - 審議 - 法律 - 議論 - 立法府 - 今回 - それ - です - わけ,0.975132,False


In [71]:
trans = Translator()

result = pd.DataFrame(allm.groupby('classes')['Name'].value_counts().groupby(level=0).nlargest(1))
result.columns = ["Count"]
result.index.names = ["0", "Committee", "Topic"]
result = result.reset_index()
result = result.iloc[:,1:4]
result["Topic_t"] = [" ".join([trans.translate(k).text for k in t.split("_")]) for t in result["Topic"]]

In [72]:
result

,Committee,Topic,Count,Topic_t
0,その他,7_学校_教員_学級_研修,478,7 School teacher class training
1,予算委員会,2_接種_ワクチン_副反応_高齢者,314,2 inoculation vaccination side reaction senior...
2,内閣委員会,1_雇用_賃上げ_賃金_最低賃金,167,1 employment wage increase wage minimum wage
3,厚生労働委員会,2_接種_ワクチン_副反応_高齢者,570,2 inoculation vaccination side reaction senior...
4,国土交通委員会,12_住宅_家賃_空き家_マンション,143,12 housing rent empty house apartment
5,外務委員会,9_ウクライナ_ロシア_侵略_侵攻,173,9 Ukraine Russia aggression invasion
6,法務委員会,25_裁判_判決_裁判官_審理,144,25 trial judgment judge trial
7,経済産業委員会,8_原発_原子力_規制_稼働,143,8 nuclear power plant nuclear power regulation...
8,総務委員会,24_nhk_受信料_放送_会長,214,24 nhk reception fee broadcast chairman
9,財務金融委員会,3_消費税_課税_税制_法人税,390,3 sale tax taxation tax system Corporate tax


In [74]:
import spacy

In [80]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [84]:
dct = {t:" ".join([trans.translate(k).text for k in t.split("_")]) for t in allm["Name"].unique()}

In [87]:
dct2 = {t:" ".join([trans.translate(k).text for k in t.split("_")]) for t in allm["committee"].unique()}

In [96]:
lst = []
for i in range(len(allm)):
    w1 = dct[allm.iloc[i,:]["Name"]]
    w1 = " ".join([w for w in w1.split()[1:]])
    w2 = " ".join(dct2[allm.iloc[i,:]["committee"]].split()[:-1])
    w1 = nlp(w1)
    w2 = nlp(w2)
    lst.append(w1.similarity(w2))

In [97]:
allmm = pd.concat([allm, pd.DataFrame({'Similarity': lst})], axis=1)

In [100]:
allmm.groupby("committee").mean("Similarity")

,Topic,Probability,Representative_document,Similarity
committee,,,,
予算委員会,33.311787,0.416290,0.036861,0.221303
予算委員会公聴会,38.417544,0.448190,0.045614,0.215531
予算委員会第一分科会,46.683721,0.467615,0.098837,0.130182
予算委員会第七分科会,53.253024,0.404869,0.042339,0.210766
予算委員会第三分科会,49.897536,0.444270,0.049287,0.225671
予算委員会第二分科会,52.801333,0.442704,0.069333,0.155307
予算委員会第五分科会,48.867308,0.460931,0.053846,0.263816
予算委員会第八分科会,45.075869,0.420994,0.048472,0.176878
予算委員会第六分科会,54.832884,0.406610,0.056604,0.281342


In [107]:
allmm[allmm["Topic"] != -1].groupby("committee").mean("Similarity").sort_values(by="Similarity", ascending=False)

,Topic,Probability,Representative_document,Similarity
committee,,,,
予算委員会第六分科会,109.180851,0.802406,0.111702,0.597883
予算委員会第七分科会,105.572277,0.795307,0.083168,0.574790
原子力問題調査特別委員会,62.780142,0.843799,0.139480,0.563250
安全保障委員会外務委員会北朝鮮による拉致問題等に関する特別委員会連合審査会,63.088235,0.827606,0.088235,0.562869
東日本大震災復興特別委員会,100.899281,0.767867,0.219424,0.556652
内閣委員会経済産業委員会連合審査会,94.837209,0.740916,0.069767,0.545607
決算行政監視委員会,73.891089,0.871664,0.064356,0.541873
内閣委員会総務委員会連合審査会,33.906977,0.787963,0.093023,0.536462
予算委員会第三分科会,94.479319,0.833412,0.092457,0.530162


In [111]:
allmm.to_csv("/content/drive/MyDrive/議事録/classsim.csv")

In [108]:
allmm["classes"] = np.where(allmm["committee"].isin(target_comm), allmm["committee"], "その他")

In [110]:
allmm[(allm["Topic"] != -1) & (allm["Topic"] != 0) & (allm["Topic"] != 5)].groupby("classes").mean("Similarity").sort_values(by="Similarity", ascending=False)

,Topic,Probability,Representative_document,Similarity
classes,,,,
その他,99.328432,0.811575,0.126198,0.441715
総務委員会,78.137913,0.795418,0.077354,0.428100
予算委員会,79.126485,0.819558,0.063614,0.416948
外務委員会,92.431939,0.763534,0.075285,0.399309
内閣委員会,88.725212,0.806578,0.073025,0.392437
国土交通委員会,82.805820,0.823792,0.074986,0.327682
財務金融委員会,81.010291,0.852811,0.071429,0.300734
農林水産委員会,89.484848,0.800675,0.081061,0.298497
厚生労働委員会,63.549633,0.833687,0.065815,0.283063


In [122]:
topic_lst = topic_model.get_topic_info()["Topic"][:60].to_list()

In [128]:
allmm[(allmm["Topic"].isin(topic_lst)) & ~ (allmm["Topic"].isin([-1,0,5,27,37,51, 59]))].groupby("classes").mean("Similarity").sort_values(by="Similarity", ascending=False)

,Topic,Probability,Representative_document,Similarity
classes,,,,
その他,21.259823,0.843131,0.022937,0.473036
総務委員会,17.236159,0.804620,0.020761,0.461861
予算委員会,18.048593,0.836545,0.012361,0.433543
内閣委員会,21.995127,0.824094,0.015701,0.399811
外務委員会,20.811453,0.777756,0.016760,0.388493
国土交通委員会,21.745005,0.855847,0.024738,0.336864
財務金融委員会,10.651685,0.897118,0.015730,0.331572
農林水産委員会,27.424855,0.836523,0.021676,0.301855
厚生労働委員会,11.608380,0.851036,0.016519,0.293755


In [134]:

print(nlp("inoculation vaccination side reaction senior citizen").similarity(nlp("Budget")))
print(nlp("inoculation vaccination side reaction senior citizen").similarity(nlp("Health and Labor")))
print(nlp("employment/wage/wage increase/minimum wage").similarity(nlp("Cabinet")))
print(nlp("Ukraine Russia aggression invasion").similarity(nlp("Foreign Affairs")))

0.4045135925692892
0.2254666118873102
0.2732807184250178
0.46551030986253467


In [76]:
allm["classes"] = np.where(allm["committee"].isin(target_comm), allm["committee"], "その他")

In [67]:
allm = allm[(allm["Topic"] != -1) & (allm["Topic"] != 0) & (allm["Topic"] != 5)]

In [ ]:
result = pd.DataFrame(allm.groupby('classes')['Name'].value_counts().groupby(level=0).nlargest(1))
result.columns = ["Count"]
result.index.names = ["0", "Committee", "Topic"]
result = result.reset_index()
result = result.iloc[:,1:3]

In [ ]:
allmm = pd.concat([allm.reset_index(drop=True),pd.Series(classes).to_frame().T],axis=1,ignore_index=True)
allmm

In [11]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)

11it [00:12,  1.11s/it]


In [16]:
topic_model.visualize_topics_per_class(topics_per_class, topics=[i for i in range(11) if i!=0 and i!=5])